In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:15

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [5]:
# Create dataframe where total_votes is greater than or equal to 20
total_votes_df = vine_df.filter("total_votes >= 20")
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [6]:
# Create new dataframe where helpful_votes/total_votes is greater than or equal to 50%
helpful_votes_df = total_votes_df.filter("helpful_votes/total_votes >= .5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [7]:
# Create a new dataframe where reviews are part of the Vine program.
vine_program_df = helpful_votes_df.filter("vine == 'Y'")
vine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R184FOUNZZ7KO8|          5|           15|         20|   Y|                N|
| R82QWN2X2OCHB|          5|          176|        208|   Y|                N|
|R1UYHBYE6790BU|          5|           44|         53|   Y|                N|
|R2J3YLX1L4EH2B|          5|          299|        321|   Y|                N|
|R3QDI539WTXKE2|          5|           26|         32|   Y|                N|
| RQTPWY6ND2NRV|          4|           37|         37|   Y|                N|
| RQZSTE0KOBY2G|          4|           98|        109|   Y|                N|
| RF2RNZEJO0K1N|          4|           23|         26|   Y|                N|
| ROB6VOW41E8O5|          4|          155|        172|   Y|                N|
|R3ASMXPEPYY28T|          3|           11|         20|   Y|     

In [8]:
# Create a new dataframe where reviews are not part of the Vine program.
not_vine_df = helpful_votes_df.filter("vine == 'N'")
not_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [12]:
# total number of reviews
total_vine = vine_program_df.count()
total_not_vine = not_vine_df.count()

In [20]:
total_vine

1080

In [21]:
total_not_vine

49673

In [15]:
# total 5 star vine reviews
five_star_vine = vine_program_df.filter("star_rating == 5").count()
five_star_vine

454

In [16]:
# total 5 star not vine reviews
five_star_not_vine = not_vine_df.filter("star_rating == 5").count()
five_star_not_vine

23043

In [17]:
# determine percentage of 5 star reviews for vine and not vine.
percent_vine = five_star_vine/total_vine *100
percent_not_vine = five_star_not_vine/total_not_vine *100

In [18]:
percent_vine

42.03703703703704

In [19]:
percent_not_vine

46.3893865882874